# Tutorial 1: Generating a training set of structures

In this tutorial, you will learn how to create a set of structures with the purpose of training a cluster exapnsion (CE) model. We start by creating a [parent lattice](https://sol.physik.hu-berlin.de/cell/modules/parent_lattice.html) object, which will be used throughout the CE construction. Examples of parent lattice creation for 3D (bulk) and 2D (surface) cases are given. The 2D example shows the multicomposition-multilattice capabilities of  [CELL](https://sol.physik.hu-berlin.de/cell/index.html). Next, you will learn to perform the generation of random structures out of [supercells](https://sol.physik.hu-berlin.de/cell/modules/super_cell.html) based on the parent lattice. These random [structures](https://sol.physik.hu-berlin.de/cell/modules/structure.html) are added to a [structures set](https://sol.physik.hu-berlin.de/cell/modules/structures_set.html) object. Such objects are extremely useful, since they allow grouping together structures for different purposes, e.g. training, testing, selection, etc. [CELL](https://sol.physik.hu-berlin.de/cell/index.html)'s structures sets also admit a representation in terms of [ASEs structure database](https://wiki.fysik.dtu.dk/ase/ase/db/db.html#module-ase.db) objects, thus inheriting all the advantages in terms of visualization, storing and retrieving structures and data in a convenient way. Excercises are proposed at the end of the tutorial.

## Building parent lattices

Here you will learn how to set up and visualize a [parent lattice](https://sol.physik.hu-berlin.de/cell/modules/parent_lattice.html), which is the most basic object in [CELL](https://sol.physik.hu-berlin.de/cell/index.html). We will consider two examples: a bulk fcc crystal, and a surface system with adsorbed atoms and surface alloying. 

We start with the example of a **bulk binary fcc** metal alloy:

In [1]:
from ase.build import bulk
from clusterx.parent_lattice import ParentLattice

pri1 = bulk('Pd', 'fcc')
sub1 = bulk('Au', 'fcc')

platt1 = ParentLattice(pri1, substitutions=[sub1])

In the first line of the code shown above, we import the `bulk` module of the [Atomic Simulation Environment (ASE)](https://wiki.fysik.dtu.dk/ase/). In the second line, the [ParentLattice](https://sol.physik.hu-berlin.de/cell/modules/parent_lattice.html)  class of **CELL** is loaded. In the next two lines, using the imported `bulk` function, we define [Atoms objects](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#module-ase.atoms) corresponding to the pristine non-substituted Cu (fcc) lattice (`pri1`) and the fully substituted Al (fcc) lattice (`sub1`). These two `Atoms` objects are then employed to initialize the `ParentLattice` object (which we call `platt1`) in the last line. 

Next, we would like to visualize the just created parent lattice. To this end, we use the `juview` function of the [visualization module](https://sol.physik.hu-berlin.de/cell/modules/visualization.html) of **CELL**:

In [2]:
from clusterx.visualization import juview

juview(platt1)

![Parent lattice CuAl](fig1-1.png) 

The figure on the left, corresponds to the pristine non-substituted Cu fcc crystal, while the figure on the right represents the fully Al-substituted crystal. In general, the function call `juview(parent_lattice)`, will generate as many additional figures as substituent species are present in the parent lattice, as you will see in the next example for a surface system.

Now, we will set up the parent lattice for a **surface system**. It consists on a fcc(111) Al surface, with possible Na substitution on the uppermost Al layer and adsorption of oxygen atoms in "on-top" configuration. This is a fictitious system which is shown here with the only purpose to illustrate possible uses of **CELL**. In order to build the parent lattice for such a system, we start with the creation of an `Atoms` object representing the pristine structure:

In [3]:
from ase.build import fcc111, add_adsorbate

pri2 = fcc111('Pd', size=(1,1,3)) # 3-atomic-layer Al slab
pri2.center(vacuum=10.0, axis=2) # add vacuum along z-axis

In the code shown above, we first load some builder utilities from ASE (`fcc111` and `add_adsorbate`). In the next three lines, we *i)* create a fcc(111) Al slab with three atomic layers; *ii)* add a vacancy (symbol `X`) site with "on top" configuration, and *iii)* add vaccuum on the sides of the slab along the $z$-direction.

In this way we have defined the pristine structure `pri2`. Now we would like to set up the substitutions: Na on the top-most Al layer and oxygen on the "on-top" vacancy sites. To proceed, we first grab some needed information from the pristine structure, as shown below: 

In [4]:
print("{0:<19s}|{1:<19s}|{2:<19s}".format("Atom index","Chemical symbol","z coordinate"))
for i, (symbol, z_coord) in enumerate(zip(pri2.get_chemical_symbols(),pri2.get_positions()[:,2])):
    print("{0:<19d}|{1:<19s}|{2:<19.3f}".format(i,symbol,z_coord))

Atom index         |Chemical symbol    |z coordinate       
0                  |Pd                 |10.000             
1                  |Pd                 |12.246             
2                  |Pd                 |14.492             


From this output, we see that the "Al" atom belonging to the uppermost layer (i.e. the Al atom with the largest $z$ coordinate) has atom index 2, and that the adsorbate vacant site (X) has *atom* index 3. With this information, we initialize the parent lattice object in an alternative way, by telling which species can occupy every atom index: This is done with the `site_symbols` argument, which allows us to tell [CELL](https://sol.physik.hu-berlin.de/cell/index.html) which atomic species can occupy every atomic site:

In [5]:
platt2 = ParentLattice(pri2, site_symbols=[['Pd','Au'],['Pd','Au'],['Pd','Au']])
juview(platt2)

![Parent lattice of complex surface](fig1-2.png) 

In this way, we see that for atom indices 0 and 1 only `['Al']` is allowed, while atom index 2 admits the species in the array `['Al','Na']` and atom index 3 admits species `['X','O']`, where `'X'` denotes a vacancy. The figures above denote: 
 - *left:* pristine non-substituted lattice with vacancy-site indicated with white color
 - *middle:* on-top vacancy site substituted by oxygen (red)
 - *right:* top-most Al layer substituted by Na (purple)

Note that this pictorial representation of the parent lattice is not meant to show all possible substitutions (this will come later with the generation of structures), but only the possible substitutions of the pristine structure (left), one by one.

## Building structure sets

In order to generate *ab initio* data to train a cluster expansion model, we need to perform calculations of the property of interest for different atomic configurations of the crystal. These are obtained by, first, creating supercells based on the parent lattice, and second, specifying the actual placement of the different species in the supercell's lattice, i.e. *decorating* the supercells. In [CELL](https://sol.physik.hu-berlin.de/cell/index.html), super cells are represented by objects of the class [SuperCell](https://sol.physik.hu-
berlin.de/cell/modules/super_cell.html). We will take the example of the surface system above, and create a $4\times4\times1$ super cell object:

In [6]:
from clusterx.super_cell import SuperCell
import numpy as np

scell2 = SuperCell(platt2,[4,4])
juview(scell2)

![Super cell](fig1-3.png) 

In the code shown above, we start by importing the [SuperCell](https://sol.physik.hu-berlin.de/cell/modules/super_cell.html) class. Then we create an object of this class and assign it to the variable `scell2`. The initialization of the super cell takes two arguments. One is the parent lattice previously created, `platt2`, and the other can be an integer, a vector of integers or a $3\times 3$ matrix with integer entries defining the transformation matrix $P$, which relates the supercell lattice vectors $S$ with the parent lattice vectors $V$: $S=PV$ (the rows of the matrices $S$ and $V$ are the cartesian coordinates of the corresponding lattice vectors, for the precise use of different input options visit the [documentation](https://sol.physik.hu-berlin.de/cell/modules/super_cell.html)). Finally, we visualize the super cell by calling the `juview` function. The interpretation of the generated figure is the same as in the case of the parent lattice explained above.

As you can see, a super cell looks very much like an enlarged parent lattice, indeed, objects of the [SuperCell](https://sol.physik.hu-berlin.de/cell/modules/super_cell.html) class inherit from the [ParentLattice](https://sol.physik.hu-berlin.de/cell/modules/parent_lattice.html) class and share many properties.

Now, using the created super cell, we will generate a few random decorations of it at different concentrations. The generated structures will be collected in a [StructuresSet](https://sol.physik.hu-berlin.de/cell/modules/structures_set.html) object.

Before doing so, however, we need to find out the sublattice definitions in the just created [SuperCell](https://sol.physik.hu-berlin.de/cell/modules/super_cell.html) object. This is necesary to correctly address the concentrations of Na substituents and oxygen atoms in the generation of random structures. This information is retrieved with the method `get_sublattice_types`:

In [7]:
scell2.get_sublattice_types(pretty_print=True)


+--------------------------------------------------------------------+
|              The structure consists of 1 sublattices               |
+--------------------------------------------------------------------+
| Sublattice type |       Chemical symbols       |  Atomic numbers   |
+--------------------------------------------------------------------+
|        0        |         ['Pd' 'Au']          |      [46 79]      |
+--------------------------------------------------------------------+



This tells us that the supercell consists of three sublattices, with indices $0$, $1$ and $2$. Lattice sites belonging to sublattice $0$, contain species `"X"` (vacancy) and can be substituted by species `"O"` (oxygen) , sites belonging to sublattice $1$ contain `"Al"` and can be substituted by `"Na"`; while sites in $2$ contain `"Al"` and can not be substituted. 

In the code shown below, we first load the [`StructuresSet`](https://sol.physik.hu-berlin.de/cell/modules/structures_set.html) class and then create a structures-set object that we call `sset2`. Next, in three different `for` loops, by using the [`gen_random()`](https://sol.physik.hu-berlin.de/cell/modules/super_cell.html) method, we create *i)* two random structures with 4 on-top oxygen atoms, *ii)* two random structures with 4 Al$\rightarrow$Na substitutions, and *iii)* two random structures with 4 oxygen atoms and 4 Al$\rightarrow$Na substitutions:

In [8]:
from clusterx.structures_set import StructuresSet

sset2 = StructuresSet(platt2)

nstruc = 6

# i) Random structures
for i in range(nstruc):
    sset2.add_structure(scell2.gen_random({0:[20]}))


juview(sset2)

![Random structures](fig1-4.png)

In the figure shown above, red spheres represent oxygen atoms and purple spheres represent Na substitutions.

## Excercise 1

Build a parent lattice for a two-dimensional square lattice of a binary (e.g. SiGe) material and create (and visualize) 6 random structures on a $5\times5$ super cell.

As help, you can use the following Atoms object to initialze the [`ParentLattice`](html/modules/parent_lattice.html) object:

In [9]:
from ase import Atoms

a=3.0
pri4 = Atoms(positions=[[0,0,0]],symbols=['Si'],cell=[[a,0,0],[0,a,0],[0,0,2*a]],pbc=(1,1,0))
pri5 = Atoms(positions=[[0,0,0]],symbols=['Ge'],cell=[[a,0,0],[0,a,0],[0,0,2*a]],pbc=(1,1,0))

platt1 = ParentLattice(pri4, substitutions=[pri5])

from clusterx.visualization import juview

juview(platt1)

pri2 = fcc111('Al', size=(1,1,3)) # 3-atomic-layer Al slab
add_adsorbate(pri2,'X',1.7,'ontop') # on-top vacancy site
pri2.center(vacuum=10.0, axis=2) # add vacuum along z-axis

print("{0:<19s}|{1:<19s}|{2:<19s}".format("Atom index","Chemical symbol","z coordinate"))
for i, (symbol, z_coord) in enumerate(zip(pri2.get_chemical_symbols(),pri2.get_positions()[:,2])):
    print("{0:<19d}|{1:<19s}|{2:<19.3f}".format(i,symbol,z_coord))

Atom index         |Chemical symbol    |z coordinate       
0                  |Al                 |10.000             
1                  |Al                 |12.338             
2                  |Al                 |14.677             
3                  |X                  |16.377             


## Excercise 2

Generate and visualize a few random structures for the fcc CuAl alloy of the first example on this tutorial. Do it so in a $3\times3\times3$ super cell.